In [ ]:
import numpy as np
import random
from scipy.stats import special_ortho_group

# Function to generate random points
def generate_random_point():
    return np.random.rand(3)
gt_rot = special_ortho_group.rvs(3)  # Random rotation matrix
gt_t = np.random.uniform(-0.5,0.5,3)  
# Generate two random points
planes = []
lines = []
origin = np.array([0, 0, 0])

for i in range(15):
    point1 = generate_random_point()
    point2 = generate_random_point()


    # Define vectors from the origin to the two points    
    plane = np.hstack([origin + np.random.uniform(-0.005,0.005,3)  , point1 + np.random.uniform(-0.005,0.005,3)  , point2+ np.random.uniform(-0.005,0.005,3)  , i])
    planes.append(plane)
    # Define two points within the plane
    # For simplicity, we take linear combinations of vec1 and vec2 with random coefficients
    rot_t_origin = gt_rot @ (origin - gt_t)
    rot_t_vec1 = gt_rot @ (point1-gt_t)
    rot_t_vec2 = gt_rot @ (point2-gt_t)

    line = np.hstack([rot_t_origin,rot_t_vec2,i])
    lines.append(line)



wrong_num = 5
for i in range(wrong_num):

    corner1 = np.random.rand(3)
    corner2 = np.random.rand(3)

    point1 = generate_random_point()
    point2 = generate_random_point()
    point3 = generate_random_point()

    plane = np.hstack([point1, point2, point3,i + 10])
    line = np.hstack([corner1,corner2,i+10])

    planes.append(plane)
    lines.append(line)

line_file = "/home/jaeho/lines.txt"
plane_file = "/home/jaeho/plane.txt"

with open(line_file, 'w') as file:

    file.write(f"{len(lines)}\n")
    for line in lines:
        file.write(f"{line[0]} {line[1]} {line[2]} {line[3]} {line[4]} {line[5]} {int(line[6])}\n")

with open(plane_file, 'w') as file:
    file.write(f"{len(planes)}\n")
    for idx,p in enumerate(planes):
        file.write(f"{p[0]} {p[1]} {p[2]} {p[3]} {p[4]} {p[5]} {p[6]} {p[7]} {p[8]} {idx}\n")

print(gt_rot, gt_t)

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def write_trajectory(filename, poses):
    with open(filename, 'w') as file:
        for timestamp, pose in poses:
            # print(pose)
            rotation = R.from_matrix(pose[:3, :3])
            q = rotation.as_quat()  # Returns (x, y, z, w)
            translation = pose[:3, 3]
            file.write(f"{timestamp} {translation[0]} {translation[1]} {translation[2]} {q[0]} {q[1]} {q[2]} {q[3]}\n")


data_dir = "/home/jaeho/office0_delta.txt"
odometry_data = np.loadtxt(data_dir)
poses = []

pose = np.eye(4)

for i in range(399):
    odometry_result = np.eye(4)
    delta_pose = odometry_data[i]
    rotation = R.from_quat([delta_pose[3], delta_pose[4], delta_pose[5], delta_pose[6]]).as_matrix()  # Quaternion to rotation matrix
    se3 = np.eye(4)
    se3[:3, :3] = rotation
    se3[:3, 3] = [delta_pose[0], delta_pose[1], delta_pose[2]]

    odometry_result[:3,:3] = rotation
    odometry_result[:3,3] = se3[:3, 3]

    pose = np.dot(pose, odometry_result)
    poses.append((0.5* (i), pose))

save_dir = '/home/jaeho/office0_bnb.txt'
write_trajectory(save_dir, poses)